In [1]:
import numpy as np
import pandas as pd
import streamlit as st
from datetime import datetime, timedelta
import os
import gspread
from google.oauth2 import service_account
from gspread_dataframe import set_with_dataframe
# from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from configs import *
from placeholder_events import *
import requests

In [2]:
#function to create api connection to google sheets
def connect_to_gs(service_account_key):
    scopes = ['https://www.googleapis.com/auth/spreadsheets']
    credentials = service_account.Credentials.from_service_account_info(service_account_key, scopes=scopes)
    gs_connection = gspread.authorize(credentials)
    return gs_connection


#create connection
gs_connection = connect_to_gs(st.secrets["gcp_service_account"])

In [4]:
#function to write data to google sheets
def write_google_sheets_data(gc, df, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        df_values = df.values.tolist()
        gs.values_append(sheet_name, {'valueInputOption': 'RAW'}, {
                    'values': df_values})

        return None

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print(f"Error: Worksheet not found, please create a new tab named:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
     

In [3]:
#function to fetch data from google sheets
def fetch_google_sheets_data(gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None
    
#fetch data


In [5]:
def fetch_max_gw(_gc, sheet_name, sheet_key):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # Convert the first column to numeric
        df[headers[0]] = pd.to_numeric(df[headers[0]])

        # Get the maximum value of the first column
        max_value = df[headers[0]].max()

        return int(max_value)

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None

In [6]:
def update(_gc):
    with requests.Session() as session:
        general_response = session.get(general_endpoint).json()

    events = pd.DataFrame(general_response['events'])[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]
    if len(events[events['is_current'] == True]) == 0:
        print('season hasn\'t started pull placeholder data')
        events = pd.DataFrame(placeholder_events)[['id', 'is_previous', 'is_current', 'is_next', 'finished', 'data_checked',]]

    
    gw = int(events[events["is_current"] == True]["id"])
    finished = bool(events[events["is_current"] == True]["finished"].values)
    data_checked = bool(events[events["is_current"] == True]["data_checked"].values)
    
    if finished and data_checked:
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        max_stored_gw = fetch_max_gw(_gc, gameweek_results_table, google_sheet_key)
        if max_stored_gw < gw:      
            print('max stored_gw < current week')
            print('we will update here')
            return True, gw
        
        else: 
            print('data is already latest - pull from gs')
            return False, max_stored_gw
    else:
        print("its either a new week or the old week hasnt completetly finished - pull from gs")
        print(f"finished: {finished}\nchecked: {data_checked}\ngame week: {gw}")
        return False, gw

In [7]:
#the result of the update function is cached for 6 hours 
update_flag, current_gw = update(gs_connection)

if update_flag == True:
    print('There is new data, google sheets will be updated now')
    #update functions
    # gameweek_results_update()     <--- get new points
    # managers_update()             <--- get new managers
    # gameweek_teams_update()       <--- get new reds/og/np
    # auto_drinks_update()          <--- get new drinks
    # st.cache_data.clear()         <--- makes sure functions below are rerun 

season hasn't started pull placeholder data
its either a new week or the old week hasnt completetly finished - pull from gs
finished: False
checked: False
game week: 1


In [8]:
# gameweek_results_update()     <--- get new points
# managers_update()             <--- get new managers
# gameweek_teams_update()       <--- get new reds/og/np
# auto_drinks_update()          <--- get new drinks
# st.cache_data.clear()         <--- makes sure functions below are rerun 
# get_illegible_nominees()

In [9]:
league_id = '353458'  # <- YNDA
general_endpoint = "https://fantasy.premierleague.com/api/bootstrap-static/"
league_endpoint = (f"https://fantasy.premierleague.com/api/leagues-classic/{league_id}/standings")
fixture_endpoint = "https://fantasy.premierleague.com/api/fixtures/"

In [10]:
# def gameweek_results_update():
    

In [11]:
with requests.Session() as session:
    general_response = session.get(general_endpoint).json()
    league_response = session.get(league_endpoint).json()
    fixture_response = session.get(fixture_endpoint).json()

In [12]:
fixture_response

[{'code': 2367552,
  'event': None,
  'finished': False,
  'finished_provisional': False,
  'id': 15,
  'kickoff_time': None,
  'minutes': 0,
  'provisional_start_time': False,
  'started': None,
  'team_a': 6,
  'team_a_score': None,
  'team_h': 12,
  'team_h_score': None,
  'stats': [],
  'team_h_difficulty': 2,
  'team_a_difficulty': 2,
  'pulse_id': 93335},
 {'code': 2367713,
  'event': None,
  'finished': False,
  'finished_provisional': False,
  'id': 176,
  'kickoff_time': None,
  'minutes': 0,
  'provisional_start_time': False,
  'started': None,
  'team_a': 4,
  'team_a_score': None,
  'team_h': 13,
  'team_h_score': None,
  'stats': [],
  'team_h_difficulty': 3,
  'team_a_difficulty': 5,
  'pulse_id': 93496},
 {'code': 2367538,
  'event': 1,
  'finished': False,
  'finished_provisional': False,
  'id': 1,
  'kickoff_time': '2023-08-11T19:00:00Z',
  'minutes': 0,
  'provisional_start_time': False,
  'started': False,
  'team_a': 13,
  'team_a_score': None,
  'team_h': 6,
  'te

In [13]:
league_respons_ph = {"league": {
    "id": 1,
    "name": "ynda"},

"new_entries":{'has_next': False,
  'page': 1,
  'results': [{'entry': 4984487,
    'entry_name': 'Briyani Bois United',
    'joined_time': '2023-08-08T07:54:19.864914Z',
    'player_first_name': 'Tijil',
    'player_last_name': 'Jugadoe'},
   {'entry': 2429570,
    'entry_name': 'Hairy Man-Gina',
    'joined_time': '2023-08-08T07:32:09.612812Z',
    'player_first_name': 'Hethe',
    'player_last_name': 'Brinkman'}]},
    "standings":{
        'has_next': False,
        'page': 1,
        'results': [{
            "id": 1234,
            "event_total": 70,
            "player_name": "Connor McDonald",
            "rank": 2,
            "last_rank": 4,
            "total": 249,
            "entry": 534613,
            "entry_name": "AllHitsNoMisses"
            },
            {
            "id": 4567,
            "event_total": 59,
            "player_name": "Alex Wietzorrek",
            "rank": 3,
            "last_rank": 5,
            "total": 235,
            "entry": 560769,
            "entry_name": "Big Mac Allister"
            }]
    }


}

In [14]:
def fetch_manager_data(_gc, sheet_name, sheet_key, columns_list):
    try:
        # Open specific sheet
        gs = _gc.open_by_key(sheet_key)

        # Open specific tab within the sheet
        tab = gs.worksheet(sheet_name)

        data = tab.get_all_values()
        headers = data.pop(0)
        df = pd.DataFrame(data, columns=headers)

        # to handle numeric columns that are imported as strings
        for column in columns_list:
            df[column] = pd.to_numeric(df[column])

        return df

    except gspread.exceptions.APIError as e:
        print("Error accessing Google Sheets API:", e)
        return None
    except gspread.exceptions.WorksheetNotFound as e:
        print("Error: Worksheet not found:", e)
        return None
    except Exception as e:
        print("An error occurred:", e)
        return None


In [15]:
def append_missing_rows(df1, df2, current_gw):
    t1, t2 = df1[["entry", "player_name"]], df2 [["entry", "player_name"]]
    missing_rows = t2[~t2['entry'].isin(t1['entry'])]
    missing_rows["event_joined"] = current_gw
    missing_rows["uno_reverse"] = "Yes"
    missing_rows
    return missing_rows

In [16]:
def managers_update(league_respons_ph, current_gw):
    #once live this will be the real way
    # with requests.Session() as session:
    #     league_response = session.get(league_endpoint).json()
    
    df2 = pd.DataFrame(league_respons_ph["standings"]["results"])
    df1 = fetch_manager_data(gs_connection, managers_table, google_sheet_key, ['entry', 'event_joined'])
    m = append_missing_rows(df1, df2, current_gw)
    write_google_sheets_data(gs_connection, m, managers_table, google_sheet_key)
    return df1[["entry", "player_name"]]

In [17]:
manager_details = managers_update(league_respons_ph, current_gw)

In [18]:
print(manager_details)

      entry        player_name
0   1742534  Niklas Wietzorrek
1   1930701      Liam Smorfitt
2   2208836       Divyam Dixit
3    552345      Marco Gouveia
4    578302        Ryan Shacks
5   2123591        Peter Wertz
6   1002132      Devon Hodgson
7   2032628     Hethe Brinkman
8   3648627        Jason Perry
9    560769    Alex Wietzorrek
10  1947717         Cole Floyd
11  1789073       Devon Jansen
12   534613    Connor McDonald


In [19]:
entry_endpoint = f'https://fantasy.premierleague.com/api/entry/{247866}/history/'
# connor = 247866
# alex = 364408

with requests.Session() as session:
    entry_response = session.get(entry_endpoint).json()


In [20]:
entry_response_ph = {'current': [
        {
        "event": 1,
        "points": 70,
        "total_points": 249,
        "rank":830350,
        "rank_sort":831979,
        "overall_rank": 416732,
        "bank": 19,
        "value": 1007,
        "event_transfers": 2,
        "event_transfer_cost": 4,
        "points_on_bench": 5
    },
        {
        "event": 2,
        "points": 70,
        "total_points": 249,
        "rank":830350,
        "rank_sort":831979,
        "overall_rank": 416732,
        "bank": 19,
        "value": 1007,
        "event_transfers": 2,
        "event_transfer_cost": 4,
        "points_on_bench": 5
    },
        {
        "event": 3,
        "points": 70,
        "total_points": 249,
        "rank":830350,
        "rank_sort":831979,
        "overall_rank": 416732,
        "bank": 19,
        "value": 1007,
        "event_transfers": 2,
        "event_transfer_cost": 4,
        "points_on_bench": 5
    },
    {
        "event": 4,
        "points": 70,
        "total_points": 249,
        "rank":830350,
        "rank_sort":831979,
        "overall_rank": 416732,
        "bank": 19,
        "value": 1007,
        "event_transfers": 2,
        "event_transfer_cost": 4,
        "points_on_bench": 5
    }
],
 'past': [{'season_name': '2021/22', 'total_points': 2401, 'rank': 306674},
  {'season_name': '2022/23', 'total_points': 2396, 'rank': 685211}],
 'chips': []}

In [21]:
for entry, player_name in zip(manager_details['entry'], manager_details['player_name']):
    print(entry)
    print(player_name)

1742534
Niklas Wietzorrek
1930701
Liam Smorfitt
2208836
Divyam Dixit
552345
Marco Gouveia
578302
Ryan Shacks
2123591
Peter Wertz
1002132
Devon Hodgson
2032628
Hethe Brinkman
3648627
Jason Perry
560769
Alex Wietzorrek
1947717
Cole Floyd
1789073
Devon Jansen
534613
Connor McDonald


In [22]:
def gameweek_results_update(_gc, current_gw, manager_details):
    new_gameweek_results = pd.DataFrame()

    for entry, player_name in zip(manager_details['entry'], manager_details['player_name']):
        entry_endpoint = f'https://fantasy.premierleague.com/api/entry/{entry}/history/'
        with requests.Session() as session:
            entry_response = session.get(entry_endpoint).json()

        current_events = entry_response['current'] #<- unhash and remove json_response argument

        # current_events = json_response['current']
        
        for event_data in current_events:
            if event_data['event'] == current_gw:
                new_player_row = {
                    'event': current_gw,
                    'entry': entry,
                    'player_name': player_name,
                    'points': event_data['points'],
                    'total_points': event_data['total_points'],
                    'event_transfers': event_data['event_transfers'],
                    'points_on_bench': event_data['points_on_bench']}

                new_gameweek_results = new_gameweek_results.append(new_player_row, ignore_index=True)
    write_google_sheets_data(_gc, new_gameweek_results, gameweek_results_table, google_sheet_key)

    # Find player_name with lowest points
    lowest_points_player = new_gameweek_results.loc[new_gameweek_results['points'].idxmin()]['player_name']
    
    return lowest_points_player



In [23]:
manager_details

,entry,player_name
0,1742534,Niklas Wietzorrek
1,1930701,Liam Smorfitt
2,2208836,Divyam Dixit
3,552345,Marco Gouveia
4,578302,Ryan Shacks
5,2123591,Peter Wertz
6,1002132,Devon Hodgson
7,2032628,Hethe Brinkman
8,3648627,Jason Perry
9,560769,Alex Wietzorrek


In [24]:
current_gw

1

In [25]:
manager_id = 534613
event_id = 1
f'https://fantasy.premierleague.com/api/entry/{manager_id}/event/{event_id}/picks/'

'https://fantasy.premierleague.com/api/entry/534613/event/1/picks/'

In [26]:
live_team_data_placeholder = {
    "active_chip": 0,
    "automatic_subs": [
        {
            "entry": 534613,
            "element_in": 60,
            "element_out": 10,
            "event": 1
        },
        {
            "entry": 534613,
            "element_in": 15,
            "element_out": 11,
            "event": 1
        }
    ],
    "entry_history": {
        "event": 1,
        "points": 70,
        "total_points": 249,
        "rank":830350,
        "rank_sort":831979,
        "overall_rank": 416732,
        "bank": 19,
        "value": 1007,
        "event_transfers": 2,
        "event_transfer_cost": 4,
        "points_on_bench": 5
    },
    "picks": [
        {
            "element": 390,
            "position": 1,
            "multiplier": 1,
            "is_captain": 0,
            "is_vice_captain": 0
        },
        {
            "element": 10,
            "position": 2,
            "multiplier": 1,
            "is_captain": 0,
            "is_vice_captain": 0
        },
        {
            "element": 11,
            "position": 2,
            "multiplier": 1,
            "is_captain": 0,
            "is_vice_captain": 0
        },
        {
            "element": 12,
            "position": 2,
            "multiplier": 1,
            "is_captain": 0,
            "is_vice_captain": 0
        }
    ]
}

In [27]:
live_gw_placeholder = {
    "elements": [
        {"id":390,
         "stats": {
             "minutes": 72,
             "goals_scored": 2,
             "assists": 1,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 1
             }
        },
        {"id":100,
         "stats": {
             "minutes": 72,
             "goals_scored": 0,
             "assists": 1,
             "own_goals": 0,
             "red_cards": 1,
             "penalties_missed": 0
             }
        },
        {"id":60,
         "stats": {
             "minutes": 72,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":10,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 1,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":15,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":12,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 1,
             "penalties_missed": 0
             }
        }
    ]
}

In [28]:
def get_player_stats(player_id, live_gw_data):
    for element in live_gw_data["elements"]:
        if element["id"] == player_id:
            return element["stats"]
    return None

In [29]:
def analyze_picks(picks, live_gw_data):
    red_card = 0
    own_goal = 0
    penalties_missed = 0

    for player_id in picks:
        player_stats = get_player_stats(player_id, live_gw_data)
        if player_stats:
            red_card += player_stats["red_cards"]
            own_goal += player_stats["own_goals"]
            penalties_missed += player_stats["penalties_missed"]

    return red_card > 0, own_goal > 0, penalties_missed > 0

In [49]:
def gameweek_results_update(_gc, current_gw, manager_details):
    new_gameweek_results = pd.DataFrame()
    picks_data = pd.DataFrame()
    for entry, player_name in zip(manager_details['entry'], manager_details['player_name']):
        live_teams_endpoint = f'https://fantasy.premierleague.com/api/entry/{entry}/event/{current_gw}/picks/'
        with requests.Session() as session:
            live_teams_response = session.get(live_teams_endpoint).json()

        live_teams = live_teams_response['entry_history']

        # current_events = live_team_data_placeholder['entry_history']
        # element_picks = live_team_data_placeholder
        
        new_player_row = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'points': live_teams['points'],
            'total_points': live_teams['total_points'],
            'event_transfers': live_teams['event_transfers'],
            'points_on_bench': live_teams['points_on_bench']}

        new_gameweek_results = pd.concat([new_gameweek_results, pd.DataFrame([new_player_row])], ignore_index=True)


        #get player pics
        elements_out = [sub["element_out"] for sub in live_teams_response["automatic_subs"]]
        elements_in = [sub["element_in"] for sub in live_teams_response["automatic_subs"]]
        picks = live_teams_response["picks"]

        element_ids_modified = [pick["element"] for pick in picks if pick["element"] not in elements_out]
        element_ids_modified.extend(elements_in)

        live_gameweek_endpoint = f'https://fantasy.premierleague.com/api/event/{current_gw}/live/'
        with requests.Session() as session:
            live_gameweek_response = session.get(live_gameweek_endpoint).json()

        # live_gameweek_response_elements = live_gameweek_response['elements'] #<- unhash and remove json_response argument
        
        # live_data = live_gw_placeholder['elements']
        red_card_flag, own_goal_flag, penalties_missed_flag = analyze_picks(element_ids_modified, live_gameweek_response)

        new_picks_data = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'picks': str(element_ids_modified),
            'red_card': red_card_flag,
            'own_goal': own_goal_flag,
            'missed_pen': penalties_missed_flag
        }

        picks_data = pd.concat([picks_data, pd.DataFrame([new_picks_data])], ignore_index=True)

    #update google sheets
    write_google_sheets_data(_gc, new_gameweek_results, gameweek_results_table, google_sheet_key)
    write_google_sheets_data(_gc, picks_data, gameweek_teams_table, google_sheet_key)

    # Find player_name with lowest points
    # lowest_points_player = new_gameweek_results.sort_values(by=['points', 'total_points', 'player_name'],ascending=[True, True, False],).iloc[0]['player_name']
    
    return None

In [ ]:
def gameweek_results_update(_gc, current_gw, manager_details):
    new_gameweek_results = pd.DataFrame()
    picks_data = pd.DataFrame()

    for entry, player_name in zip(manager_details['entry'], manager_details['player_name']):
        live_teams_endpoint = f'https://fantasy.premierleague.com/api/entry/{entry}/event/{current_gw}/picks/'
        with requests.Session() as session:
            live_teams_response = session.get(live_teams_endpoint).json()

        live_teams = live_teams_response['entry_history']

        # Create new player row
        new_player_row = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'points': live_teams['points'],
            'total_points': live_teams['total_points'],
            'event_transfers': live_teams['event_transfers'],
            'points_on_bench': live_teams['points_on_bench']}
        
        # Append to new_gameweek_results using concat
        new_gameweek_results = pd.concat([new_gameweek_results, pd.DataFrame([new_player_row])], ignore_index=True)

        # Get player picks
        elements_out = [sub["element_out"] for sub in live_teams_response["automatic_subs"]]
        elements_in = [sub["element_in"] for sub in live_teams_response["automatic_subs"]]
        picks = live_teams_response["picks"]

        element_ids_modified = [pick["element"] for pick in picks if pick["element"] not in elements_out]
        element_ids_modified.extend(elements_in)

        live_gameweek_endpoint = f'https://fantasy.premierleague.com/api/event/{current_gw}/live/'
        with requests.Session() as session:
            live_gameweek_response = session.get(live_gameweek_endpoint).json()

        # Analyze player picks
        red_card_flag, own_goal_flag, penalties_missed_flag = analyze_picks(element_ids_modified, live_gameweek_response)

        new_picks_data = {
            'event': current_gw,
            'entry': entry,
            'player_name': player_name,
            'picks': str(element_ids_modified),
            'red_card': red_card_flag,
            'own_goal': own_goal_flag,
            'missed_pen': penalties_missed_flag
        }

        # Append to picks_data using concat
        picks_data = pd.concat([picks_data, pd.DataFrame([new_picks_data])], ignore_index=True)

    # Update google sheets for new_gameweek_results and picks_data
    write_google_sheets_data(_gc, new_gameweek_results, gameweek_results_table, google_sheet_key)
    write_google_sheets_data(_gc, picks_data, gameweek_teams_table, google_sheet_key)
    
    return None


In [50]:
gameweek_results_update(gs_connection, current_gw, manager_details)

In [ ]:
live_gw_placeholder = {
    "elements": [
        {"id":390,
         "stats": {
             "minutes": 72,
             "goals_scored": 2,
             "assists": 1,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 1
             }
        },
        {"id":100,
         "stats": {
             "minutes": 72,
             "goals_scored": 0,
             "assists": 1,
             "own_goals": 0,
             "red_cards": 1,
             "penalties_missed": 0
             }
        },
        {"id":60,
         "stats": {
             "minutes": 72,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":10,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 1,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":15,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 0,
             "penalties_missed": 0
             }
        },
        {"id":12,
         "stats": {
             "minutes": 0,
             "goals_scored": 0,
             "assists": 0,
             "own_goals": 0,
             "red_cards": 1,
             "penalties_missed": 0
             }
        }
    ]
}

In [ ]:
# Function to filter player names based on conditions
def get_players_by_condition(df, condition_column):
    filtered_players = df[df[condition_column] == 'TRUE']
    return list(filtered_players['player_name'])

In [ ]:
def auto_assign_drinks(_gc, gameweek_results_table, gameweek_teams_table, google_sheet_key, current_gw):
    gameweek_results_table_df = fetch_google_sheets_data(_gc, gameweek_results_table, google_sheet_key,  ["event", "points", "total_points", "event_transfers_cost", "points_on_bench"])
    gameweek_teams_table_df = fetch_google_sheets_data(_gc, gameweek_teams_table, google_sheet_key,  ["event"])

    filtered_results = gameweek_results_table_df[gameweek_results_table_df["event"] == current_gw]
    filtered_teams = gameweek_teams_table_df[gameweek_teams_table_df["event"] == current_gw]

    lowest_points_player = filtered_results.sort_values(by=['points', 'total_points', 'player_name']).iloc[0]['player_name']
    created_date = (datetime.now() + timedelta(hours=2)).strftime("%d/%m/%y %H:%M:%S")
    deadline_date = (datetime.now() + timedelta(hours=2) + timedelta(days=7)).strftime("%d/%m/%y %H:00")

    # Get player names with red cards, own goals, and missed penalties
    red_card_players = get_players_by_condition(filtered_teams, 'red_card')
    own_goal_players = get_players_by_condition(filtered_teams, 'own_goal')
    missed_pen_players = get_players_by_condition(filtered_teams, 'missed_pen')

    red_card_drinks_data = {
        "event": [current_gw]*len(red_card_players),
        "nominator_name": ["auto"]*len(red_card_players),
        "drinker_name": red_card_players,
        "drink_type": ["red card"]*len(red_card_players),
        "nomination_created_date": [created_date]*len(red_card_players),
        "nomination_deadline_date": [deadline_date]*len(red_card_players),
        "nomination_completed_date": ['Not Completed']*len(missed_pen_players),
    }

    own_goal_players_drinks_data = {
        "event": [current_gw]*len(own_goal_players),
        "nominator_name": ["auto"]*len(own_goal_players),
        "drinker_name": own_goal_players,
        "drink_type": ["own goal"]*len(own_goal_players),
        "nomination_created_date": [created_date]*len(own_goal_players),
        "nomination_deadline_date": [deadline_date]*len(own_goal_players),
        "nomination_completed_date": ['Not Completed']*len(missed_pen_players),
    }

    missed_pen_players_drinks_data = {
        "event": [current_gw]*len(missed_pen_players),
        "nominator_name": ["auto"]*len(missed_pen_players),
        "drinker_name": missed_pen_players,
        "drink_type": ["missed pen"]*len(missed_pen_players),
        "nomination_created_date": [created_date]*len(missed_pen_players),
        "nomination_deadline_date": [deadline_date]*len(missed_pen_players),
        "nomination_completed_date": ['Not Completed']*len(missed_pen_players),
    }

    df1 = pd.DataFrame(red_card_drinks_data)
    df2 = pd.DataFrame(own_goal_players_drinks_data)
    df3 = pd.DataFrame(missed_pen_players_drinks_data)

    concatenated_df = pd.concat([df1, df2, df3], ignore_index=True)

    write_google_sheets_data(_gc, concatenated_df, drinks_table, google_sheet_key)
    return None



In [8]:
current_gw =1

In [36]:
_gc = gs_connection
gameweek_results_table_df = fetch_google_sheets_data(_gc, gameweek_results_table, google_sheet_key,  ["event", "points", "total_points", "event_transfers_cost", "points_on_bench"])
gameweek_teams_table_df = fetch_google_sheets_data(_gc, gameweek_teams_table, google_sheet_key,  ["event"])

filtered_results = gameweek_results_table_df[gameweek_results_table_df["event"] == current_gw]
filtered_teams = gameweek_teams_table_df[gameweek_teams_table_df["event"] == current_gw]

lowest_points_player = filtered_results.sort_values(by=['points', 'total_points', 'player_name']).iloc[0]['player_name']
created_date = (datetime.now() + timedelta(hours=2)).strftime("%d/%m/%y %H:%M:%S")
deadline_date = (datetime.now() + timedelta(hours=2) + timedelta(days=7)).strftime("%d/%m/%y %H:00")

# Get player names with red cards, own goals, and missed penalties
red_card_players = get_players_by_condition(filtered_teams, 'red_card')
own_goal_players = get_players_by_condition(filtered_teams, 'own_goal')
missed_pen_players = get_players_by_condition(filtered_teams, 'missed_pen')

red_card_drinks_data = {
    "event": [current_gw]*len(red_card_players),
    "nominator_name": ["auto"]*len(red_card_players),
    "drinker_name": red_card_players,
    "drink_type": ["red card"]*len(red_card_players),
    "nomination_created_date": [created_date]*len(red_card_players),
    "nomination_deadline_date": [deadline_date]*len(red_card_players),
    "nomination_completed_date": ["Not Completed"]*len(red_card_players),
}

own_goal_players_drinks_data = {
    "event": [current_gw]*len(own_goal_players),
    "nominator_name": ["auto"]*len(own_goal_players),
    "drinker_name": own_goal_players,
    "drink_type": ["own goal"]*len(own_goal_players),
    "nomination_created_date": [created_date]*len(own_goal_players),
    "nomination_deadline_date": [deadline_date]*len(own_goal_players),
    "nomination_completed_date": ["Not Completed"]*len(own_goal_players),
}

missed_pen_players_drinks_data = {
    "event": [current_gw]*len(missed_pen_players),
    "nominator_name": ["auto"]*len(missed_pen_players),
    "drinker_name": missed_pen_players,
    "drink_type": ["missed pen"]*len(missed_pen_players),
    "nomination_created_date": [created_date]*len(missed_pen_players),
    "nomination_deadline_date": [deadline_date]*len(missed_pen_players),
    "nomination_completed_date": ["Not Completed"]*len(missed_pen_players),
}

df1 = pd.DataFrame(red_card_drinks_data)
df2 = pd.DataFrame(own_goal_players_drinks_data)
df3 = pd.DataFrame(missed_pen_players_drinks_data)

concatenated_df = pd.concat([df1, df2, df3], ignore_index=True)

write_google_sheets_data(_gc, concatenated_df, drinks_table, google_sheet_key)

In [31]:
concatenated_df

,event,nominator_name,drinker_name,drink_type,nomination_created_date,nomination_deadline_date
0,1,auto,Marco Gouveia,red card,09/08/23 22:21:05,16/08/23 22:00
1,1,auto,Cole Floyd,red card,09/08/23 22:21:05,16/08/23 22:00
2,1,auto,Connor McDonald,red card,09/08/23 22:21:05,16/08/23 22:00
3,1,auto,Devon Hodgson,own goal,09/08/23 22:21:05,16/08/23 22:00
4,1,auto,Connor McDonald,own goal,09/08/23 22:21:05,16/08/23 22:00
5,1,auto,Cole Floyd,missed pen,09/08/23 22:21:05,16/08/23 22:00
6,1,auto,Connor McDonald,missed pen,09/08/23 22:21:05,16/08/23 22:00
